In [ ]:
from app import *
import pyperclip as clp

window_width = 1920
window_height = 1080

top_banner_height = 36

hao_height = 90
hao_width = 260

one_page_hao_num = 10

article_2rd_width = 260
article_2rd_height = 122

h_sou = 120
v_sou = 44

h_ding = 108
v_ding = 142

h_ding2 = 110
v_ding2 = 110

h_r_top = 40
v_r_top = 70

r_menu_item_height = 34 # 点开右上方三个点、出来的菜单、单项的高度

def re_open_dingyuehao():
    pag.hotkey('ctrl','alt', 'w')
    time.sleep(0.2)
    pag.hotkey('esc')
    time.sleep(0.2)
    pag.hotkey('esc')
    time.sleep(0.2)
    pag.hotkey('esc')
    time.sleep(0.2)
    pag.hotkey('ctrl','alt', 'w')
    pag.hotkey('win', 'up')
    pag.moveTo(x = h_sou, y = v_sou)
    pag.click()
    clp.copy("订阅号")
    time.sleep(0.2)
    pag.hotkey('ctrl', 'a')
    time.sleep(0.2)
    pag.hotkey('ctrl', 'v')
    pag.moveTo(x = h_ding, y = v_ding)
    pag.click()
    pag.moveTo(x = h_ding2, y = v_ding2)
    pag.click(clicks=2, interval=0.1, button='right')
    pag.moveTo(x = window_width /2 , y = window_height / 2)
    pag.hotkey('win', 'up')

def locate_hao(i=0):
    foobar = hao_height / 2
    pag.moveTo(x = foobar, y = top_banner_height + foobar+ i*hao_height)
    if i == 0:
        pag.scroll(10000000)
    time.sleep(0.3)
    pag.click()

def copy_browser_url():
    time.sleep(1.2)
    pag.hotkey('ctrl', 'l') 
    pag.hotkey('ctrl', 'c') 
    url = clp.paste()
    time.sleep(0.3)
    pag.hotkey('ctrl', 'w') 
    return url

def open_curr_article_to_browser():
    pag.moveTo(x = window_width - h_r_top, y = v_r_top)
    pag.click()
    pag.moveTo(x = window_width - h_r_top - 10, y =v_r_top+r_menu_item_height *3.5)
    pag.click()
    
# open_curr_article_to_browser()

In [ ]:
from openai_tools import *
from pachong2 import *
from db import SqliteDb

def fetch_url_ctt(url):
    print(url)
    if db.check_url(url):
        hao, title, desc, abst, ctt = db.read('article', columns=' hao, title, desc, abst, ctt ', where=f"url='{url}'")[0]
    else:
        hao, title, desc, ctt = get_wechat_artile_content(url)
        if hao is None:
            return None, None, None, None, None
        abst = ''
        if ctt:
            for i in range(4):
                try:
                    _, abst = call_with_sys(sys_prompt, ctt[:6500], 512)
                    if abst:
                        time.sleep(20)
                        break
                except openai.error.APIConnectionError:
                    print('except error.APIConnectionError, sleep...')
                    time.sleep(3)
        db.create('article', {
            'title': title,
            'hao': hao,
            'url': url,
            'desc': desc,
            'abst': abst,
            'ctt': ctt
        }) 
    print("%s : %s" % (hao, title))
    return hao, title, desc, abst, ctt

def write2file(i, hao, title, url, desc, abst, ctt):
    if i == 0:
        f_meta.write("## %s \n\n" % hao)
        f_meta.flush()
    f_meta.write("### %s \n\n %s\n\n %s\n\n" % (title, abst, url))
    f_meta.flush()

    f_ctt.write("%s : %s \n %s\n%s\n\n" % (hao, title, desc, url))
    f_ctt.write("%s\n\n" % abst)
    f_ctt.write("%s\n\n\n\n\n\n" % ctt)
    f_ctt.flush()

def article_click(i):
    pag.move(hao_width + top_banner_height, 200 + i * article_2rd_height)
    
def click_curr_hao_all_article(num):
    i = -1
    while True:
        i += 1
        if i < num:
            article_click(i)
#             moveby(my1v, hao_width + 30, top_banner_height + 400 + i * article_2rd_height)
            pag.click()
            time.sleep(1.3)
            open_curr_article_to_browser()
            url = copy_browser_url()
            if url not in all_ctt:
#                 hao, title, desc, abst, ctt = fetch_url_ctt(url)
                all_ctt[url] = url
#                 if hao is not None:
#                     write2file(i, hao, title, url, desc, abst, ctt)
        else:
            break
    f_meta.write("<hr/>\n\n\n\n\n\n")
    f_meta.flush()

# open_curr_article_to_browser()

In [ ]:
def wechat_article_list(hao_num, art_num):
    for i in range(hao_num + 1):
#         my1v=find_my1()
        if i == 0:
            locate_hao()
        elif 1 <= i <= one_page_hao_num:
#             moveby(my1v, h_first_center, v_first_center + hao_height * i)
            locate_hao(i)
            pag.click()
        else:
#             moveby(my1v, h_first_center, v_first_center + hao_height * 9)
            locate_hao(one_page_hao_num)
            pag.click()
            pag.hotkey('down')
        time.sleep(0.3)
        click_curr_hao_all_article(art_num)

if __name__ == "__main__":
    sys_prompt = "跟据用户给出的文章提取80-120字的摘要，具体时间、人物等信息不可以省略。"
    while True:
        db = SqliteDb('test.sqlite')
        db.create_table('article', 'hao TEXT, title TEXT, url TEXT, desc TEXT, abst TEXT, ctt TEXT')

        now = datetime.now().strftime('%Y%m%d_%H%M%S')
        f_meta = open("_wechat_url" + "_" + now + ".md", "w", encoding="utf-8")
        f_ctt = open("_wechat_content" + "_" + now + ".txt", "w", encoding="utf-8")

        all_ctt = {}
        wechat_article_list(12, 5)
        f_meta.close()
        f_ctt.close()
        db.close()
        
        time.sleep(10 * 60)